1.Денормализуйте таблицу так, чтобы не нужно было для каждого рекламодателя постоянно подсчитывать количество кампаний и продаж.

In [1]:
import init_spark_env # Подтягиваю файл настроек связки юпитра и спарка

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct

In [3]:
# Создаю Spark сессию
spark = SparkSession.builder \
    .appName("denormal_hw3") \
    .getOrCreate()

In [4]:
spark # проверка состояния спарка

In [16]:
# Рекламодатели
advertisers_data = [
    (1, 'Advertiser 1'),
    (2, 'Advertiser 2'),
    (3, 'Advertiser 3')
]
advertisers = spark.createDataFrame(advertisers_data, ["UniqueID", "Name"])

advertisers.show()

+--------+------------+
|UniqueID|        Name|
+--------+------------+
|       1|Advertiser 1|
|       2|Advertiser 2|
|       3|Advertiser 3|
+--------+------------+



In [17]:
# "Кoмпании"
campaigns_data = [
    (1, 'Campaign 1'),
    (2, 'Campaign 2'),
    (3, 'Campaign 3'),
    (4, 'Campaign 4')
]
campaigns = spark.createDataFrame(campaigns_data, ["UniqueID", "campaign_name"])

campaigns.show()

+--------+-------------+
|UniqueID|campaign_name|
+--------+-------------+
|       1|   Campaign 1|
|       2|   Campaign 2|
|       3|   Campaign 3|
|       4|   Campaign 4|
+--------+-------------+



In [7]:
# Продажи
sales_data = [
    (1, 1, 1),
    (2, 1, 2),
    (3, 2, 3),
    (4, 3, 4),
    (5, 3, 4)
]
sales = spark.createDataFrame(sales_data, ["UniqueID", "advertise_id", "campaign_id"])

sales.show()

+--------+------------+-----------+
|UniqueID|advertise_id|campaign_id|
+--------+------------+-----------+
|       1|           1|          1|
|       2|           1|          2|
|       3|           2|          3|
|       4|           3|          4|
|       5|           3|          4|
+--------+------------+-----------+



In [14]:
# Kоличество уникальных кампаний и количество продаж для каждого рекламодателя
campaign_counts = sales.groupBy("advertise_id").agg(countDistinct("campaign_id").alias("campaign_count"))
sales_counts = sales.groupBy("advertise_id").count().withColumnRenamed("count", "sales_count")

In [15]:
denormalized_data = advertisers.join(campaign_counts, advertisers.UniqueID == campaign_counts.advertise_id, "left") \
                               .join(sales_counts, advertisers.UniqueID == sales_counts.advertise_id, "left") \
                               .drop("advertise_id")

In [18]:
denormalized_data = denormalized_data.fillna(0)
denormalized_data.show()

+--------+------------+--------------+-----------+
|UniqueID|        Name|campaign_count|sales_count|
+--------+------------+--------------+-----------+
|       1|Advertiser 1|             2|          2|
|       2|Advertiser 2|             1|          1|
|       3|Advertiser 3|             1|          2|
+--------+------------+--------------+-----------+



2.В базе данных есть две таблицы: страны и клиенты. Одной из потребностей компании является исследование клиентов и стран с точки зрения эффективности продаж, поэтому часто выполняются объединения между таблицами: клиенты и страны. Что нужно сделать, чтобы ограничить частое объединение этих двух таблиц?

In [19]:
#  Страны
countries_data = [
    (1, 'USA'),
    (2, 'Canada'),
    (3, 'France')
]
countries = spark.createDataFrame(countries_data, ["UniqueID", "country_name"])

countries.show()

+--------+------------+
|UniqueID|country_name|
+--------+------------+
|       1|         USA|
|       2|      Canada|
|       3|      France|
+--------+------------+



In [21]:
# Клиенты
customers_data = [
    (1, 'Barac Obama', 1),
    (2, 'Mykki Maus', 2),
    (3, 'Bob Dillan', 3),
    (4, 'Donya Tramp', 1)
]
customers = spark.createDataFrame(customers_data, ["UniqueID", "customer_name", "country_id"])

customers.show()

+--------+-------------+----------+
|UniqueID|customer_name|country_id|
+--------+-------------+----------+
|       1|  Barac Obama|         1|
|       2|   Mykki Maus|         2|
|       3|   Bob Dillan|         3|
|       4|  Donya Tramp|         1|
+--------+-------------+----------+



In [26]:
denormalized_data = customers.join(countries, customers.country_id == countries.UniqueID, "left") \
                             .drop("country_id") \
                             .withColumnRenamed("country_name", "customer_country")

In [27]:
denormalized_data.show()

+--------+-------------+--------+----------------+
|UniqueID|customer_name|UniqueID|customer_country|
+--------+-------------+--------+----------------+
|       1|  Barac Obama|       1|             USA|
|       2|   Mykki Maus|       2|          Canada|
|       3|   Bob Dillan|       3|          France|
|       4|  Donya Tramp|       1|             USA|
+--------+-------------+--------+----------------+



3.Вернемся к первому примеру. Предположим, компания хочет регулярно извлекать данные о продажах, например, о кампаниях или рекламодателях с полными именами. Как мы можем решить проблему постоянной необходимости объединения таблиц?

In [28]:
# Рекламодатели
advertisers_data = [
    (1, 'Advertiser A'),
    (2, 'Advertiser B'),
    (3, 'Advertiser C')
]
advertisers = spark.createDataFrame(advertisers_data, ["UniqueID", "Name"])

advertisers.show()

+--------+------------+
|UniqueID|        Name|
+--------+------------+
|       1|Advertiser A|
|       2|Advertiser B|
|       3|Advertiser C|
+--------+------------+



In [29]:
# Кампании
campaigns_data = [
    (1, 'Campaign 1'),
    (2, 'Campaign 2'),
    (3, 'Campaign 3'),
    (4, 'Campaign 4')
]
campaigns = spark.createDataFrame(campaigns_data, ["UniqueID", "campaign_name"])

campaigns.show()

+--------+-------------+
|UniqueID|campaign_name|
+--------+-------------+
|       1|   Campaign 1|
|       2|   Campaign 2|
|       3|   Campaign 3|
|       4|   Campaign 4|
+--------+-------------+



In [30]:
# Продажи
sales_data = [
    (1, 1, 1),
    (2, 1, 2),
    (3, 2, 3),
    (4, 3, 4),
    (5, 3, 4)
]
sales = spark.createDataFrame(sales_data, ["UniqueID", "advertise_id", "campaign_id"])

sales.show()

+--------+------------+-----------+
|UniqueID|advertise_id|campaign_id|
+--------+------------+-----------+
|       1|           1|          1|
|       2|           1|          2|
|       3|           2|          3|
|       4|           3|          4|
|       5|           3|          4|
+--------+------------+-----------+



In [35]:
denormalized_sales = sales.join(advertisers, sales.advertise_id == advertisers.UniqueID, "left") \
                          .join(campaigns, sales.campaign_id == campaigns.UniqueID, "left") \
                          .drop(advertisers.UniqueID) \
                          .drop(campaigns.UniqueID) \
                          .withColumnRenamed("Name", "advertiser_name") \
                          .withColumnRenamed("campaign_name", "campaign_name")

In [36]:
denormalized_sales.show()

+--------+------------+-----------+---------------+-------------+
|UniqueID|advertise_id|campaign_id|advertiser_name|campaign_name|
+--------+------------+-----------+---------------+-------------+
|       1|           1|          1|   Advertiser A|   Campaign 1|
|       3|           2|          3|   Advertiser B|   Campaign 3|
|       2|           1|          2|   Advertiser A|   Campaign 2|
|       4|           3|          4|   Advertiser C|   Campaign 4|
|       5|           3|          4|   Advertiser C|   Campaign 4|
+--------+------------+-----------+---------------+-------------+



In [37]:
spark.stop()